In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
from scipy.stats import norm
import requests
from io import StringIO
import seaborn as sns; sns.set()
import warnings
import yfinance as yf

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

## 리스크관리 팀의 목표

1. 세미나 과정에서 학습한 내용을 바탕으로 현재 펀드가 보유중인 포트폴리오의 리스크를 측정
2. 1의 과정을 통해 운용팀의 운용 상황이 너무 높은 리스크를 감수하고 있지 않은지 확인 후 경고하거나 적절한 헤지 방법을 제안


## 리스크관리 팀의 활동 방법

주식과 같은 자산을 다룰 것이므로 신용리스크는 제한적일 것이라 판단하였습니다. 따라서 유동성리스크와 시장리스크를 기반으로 리스크를 측정할 것 입니다.
사용하는 모델로는 VaR, CVaR, 유동성 조정 CVaR을 계산할 것이며, 산출하는 방법론은 몬테카를로 시뮬레이션, 역사적 시뮬레이션, 공분산 기법입니다

후에 더 발전된 시뮬레이션 방법을 학습하거나 시장미시구조 기반 유동성 리스크 측정 기법을 학습 시 모델을 개선할 예정입니다.

### 사용 모델 : VaR, CVaR, 유동성 조정 CVaR

최대 예상 손실은 주어진 시간과 사전 정의된 신뢰구간 동안 회사가 가지고 있는 자산의 최대 예상 손실을 측정하는데 사용되는 시장리스크 관리 지표입니다. 이는 $$VaR : 1-\alpha= \int_{-\infty} ^{-var} f(x)dx $$

을 만족하는 VaR을 의미합니다.


최대 손실 평균, ES는 VaR을 초과하는 손실의 조건부 평균입니다. 이는 수익률 $x$의 확률분포함수를 $f(x)$라고 할 때,

$$ES = CVar =   -\frac{1}{1-\alpha} \int_{-\infty} ^{-var} xf(x)dx $$

로 계산할 수 있으며, $\alpha$는 신뢰수준입니다.

유동성 조정 CVaR은 유동성 지표를 반영하여 시장리스크와 유동성 리스크 사이의 상호관계를 고려한 리스크지표이며, 다음과 같이 ES 계산식을 수정하여 계산 가능합니다.

$$ES_L=ES+\mathtt{유동성비용} \newline
ES_L=\frac{1}{1-\alpha} \int_{\alpha}^1 VaR_u \ du + \frac{1}{2}P_{last}(\mu + k\sigma)$$

여기서 $P_{last}$는 주식 종가, $\mu$는 스프레드 평균, $k$는 두터운 꼬리를 수용하기 위한 스케일링 팩터, $\sigma$는 스프레드의 표준편차를 의미합니다.


### 사용 방법론 : 분산-공분산 기법, 역사적 시뮬레이션, 몬테 카를로 시뮬레이션

#### 1. Variance-Covariance Method

분산 - 공분산 기법은 다른 말로 델타-노말 기법이라고도 합니다. 이는 관측값의 분포(ex : 주식 수익률의 분포, 이자율의 분포, 기초자산의 분포 등)가 정규분포를 따른다는 것과 기초자산과 포트폴리오 가치의 관계가 선형임을 가정하고 VaR을 계산하는 것을 의미합니다. 따라서 이를 모수적 방법이라고 부르기도 합니다. 일반적으로 과거 데이터를 사용해 공분산 행렬, 평균 벡터 등을 계산합니다.

In [3]:
symbols = ["IBM", "MSFT", "INTC"]
stock3 = []
for symbol in symbols:
    stock3.append(yf.download(symbol, '2020-01-01', '2020-12-31')['Close'])
stocks = pd.DataFrame(stock3).T
stocks.columns = symbols

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
stocks.head()

,IBM,MSFT,INTC
Date,,,
2020-01-02,129.464630,160.619995,60.840000
2020-01-03,128.432129,158.619995,60.099998
2020-01-06,128.202682,159.029999,59.930000
2020-01-07,128.288712,157.580002,58.930000
2020-01-08,129.359467,160.089996,58.970001


In [5]:
stocks_returns = (np.log(stocks) - np.log(stocks.shift(1))).dropna()
stocks_returns

,IBM,MSFT,INTC
Date,,,
2020-01-03,-0.008007,-0.012530,-0.012238
2020-01-06,-0.001788,0.002581,-0.002833
2020-01-07,0.000671,-0.009160,-0.016827
2020-01-08,0.008312,0.015803,0.000679
2020-01-09,0.010513,0.012416,0.005580
...,...,...,...
2020-12-23,0.002343,-0.013125,0.008626
2020-12-24,0.006356,0.007797,0.010679
2020-12-28,0.001042,0.009873,0.000000


In [6]:
stocks_returns_mean = stocks_returns.mean()
weights  = [1/3]*3
weights /= np.sum(weights)
cov_var = stocks_returns.cov()
port_std = np.sqrt(weights.T.dot(cov_var).dot(weights))
stock_cov=np.diag(cov_var)   # 각 종목 분산 데이터

In [7]:
port_return_mean = np.dot(stocks_returns, weights).mean()

In [8]:
initial_investment = 1e6   # 원금 100만 달러에 대해 동일비중 포트폴리오와 각 종목에만 투자하는 세가지 포트폴리오 비교
conf_level = 0.95

In [9]:
def VaR_parametric(initial_investment, conf_level, cov):
    alpha = norm.ppf(1 - conf_level, stocks_returns_mean, port_std)
    for i, j in zip(stocks.columns, range(len(stocks.columns))):
        VaR_param = -initial_investment * (alpha)[j]
        print("Parametric VaR result for {} is {} ".format(i, VaR_param))
    VaR_param = -initial_investment * (alpha)
    print('--' * 25)
    return VaR_param

In [10]:
VaR_param = VaR_parametric(initial_investment, conf_level, stock_cov)

alpha = norm.ppf(1 - conf_level, port_return_mean, port_std)
VaR_param_port = (initial_investment - initial_investment * (1 + alpha))
print("Parametric VaR result for {} is {} ".format("equal_weight_portfolio", VaR_param_port))

Parametric VaR result for IBM is 42971.68071825573 
Parametric VaR result for MSFT is 41347.956059765645 
Parametric VaR result for INTC is 43514.23432115943 
--------------------------------------------------
Parametric VaR result for equal_weight_portfolio is 42611.290366393514 


#### 2.Historical Simulation VaR

역사적 시뮬레이션 방법은 분포를 가정하지 않는다는 점에서 비모수적 방법이라고도 불립니다. 역사적 시뮬레이션은 과거 관측치에서 해당 신뢰구간에 맞는 백분위수를 찾은 뒤, 이 백분위수에 초기 투자를 곱하는 방법으로 VaR을 계산합니다. 현재 비즈니스 사이클을 고려하여 과거 3년 데이터를 바탕으로 역사적 시뮬레이션을 수행할 예정입니다.


In [12]:
def VaR_historical(initial_investment, conf_level):
    Hist_percentile95 = []
    for i, j in zip(stocks_returns.columns,
                    range(len(stocks_returns.columns))):
        Hist_percentile95.append(np.percentile(stocks_returns.loc[:, i],
                                             5))
        print("Based on historical values 95% of {}'s return is {:.4f}"
             .format(i, Hist_percentile95[j]))
        VaR_historical = (initial_investment - initial_investment *
                         (1 + Hist_percentile95[j]))
        print("Historical VaR result for {} is {:.2f} "
             .format(i, VaR_historical))
        print('--' * 35)

In [13]:
VaR_historical(initial_investment,conf_level)

Based on historical values 95% of IBM's return is -0.0372
Historical VaR result for IBM is 37201.07 
----------------------------------------------------------------------
Based on historical values 95% of MSFT's return is -0.0426
Historical VaR result for MSFT is 42622.24 
----------------------------------------------------------------------
Based on historical values 95% of INTC's return is -0.0425
Historical VaR result for INTC is 42509.24 
----------------------------------------------------------------------


#### 3. Monte Carlo VaR

몬테 카를로 시뮬레이션을 위해 사용 계획 중인 주가 분포 시나리오는 기하브라운 운동(GBM)입니다. 즉

$$ {dS \over S}  = rdt + \sigma dW $$

의 확률과정을 사용할 예정입니다. 이때 $r$은 무위험이자율, $\sigma$는 주가의 $dt$시간 동안의 변동성, $dW$는 표준정규분포를 따르는 확률변수이다.

또한 포트폴리오 자산 간에는 상관관계가 존재하므로, 상관관계를 가지는 여러 개의 자산을 분석할 때에는 단순히 두 개의 기하브라운 식을 구축하는 것으로는 충분하지 않습니다. 이를 해결하기 위해 촐레츠키 분해를 적용해 몬테 카를로 시뮬레이션을 수행할 에정입니다.



In [14]:
import scipy

T = 1
days = 250
nsimulation = 300000
r=0.01
stock_sigma=np.sqrt(stock_cov)  # 각 종목의 일별 변동성

L=scipy.linalg.cholesky(np.corrcoef([stocks_returns.iloc[:, 0], stocks_returns.iloc[:, 1], stocks_returns.iloc[:, 2]]), lower=True)

Normal1 = np.random.normal(size=(T*days, nsimulation))
Normal2 = np.random.normal(size=(T*days, nsimulation))
Normal3 = np.random.normal(size=(T*days, nsimulation))

nor1 = Normal1.reshape(1,T*days*nsimulation)
nor2 = Normal2.reshape(1,T*days*nsimulation)
nor3 = Normal3.reshape(1,T*days*nsimulation)

dz=[]
for i in range(len(nor1)):
    dz.append(np.dot(L, np.array([nor1[i], nor2[i], nor3[i]])))

dz=np.array(dz[0])

cor1 = dz[0, :]
cor2 = dz[1, :]
cor3 = dz[2, :]

Normal1 = cor1.reshape(T*days,nsimulation)
Normal2 = cor2.reshape(T*days,nsimulation)
Normal3 = cor3.reshape(T*days,nsimulation)


dt = 1/250
logds1 = r*dt + Normal1 * stock_sigma[0]
logds2 = r*dt + Normal2 * stock_sigma[1]
logds3 = r*dt + Normal3 * stock_sigma[2]

path1 = np.zeros((T*days+1,nsimulation))
path2 = np.zeros((T*days+1,nsimulation))
path3= np.zeros((T*days+1,nsimulation))

path1[0] = 129.46463
path2[0] = 160.619995
path3[0] = 60.840000

for i in range(1,T*days+1):
    path1[i] = path1[i-1]*(1+logds1[i-1])  # 이미 일별 변동성을 계산했으므로 np.sqrt(dt)를 안곱해줘도 됨!
    path2[i] = path2[i-1]*(1+logds2[i-1])
    path3[i]= path3[i-1]*(1+logds3[i-1])

path1=pd.DataFrame(path1)
IBM_return=(np.log(path1)-np.log(path1.shift(1))).dropna()
path2=pd.DataFrame(path2)
MSFT_return=(np.log(path2)-np.log(path2.shift(1))).dropna()
path3=pd.DataFrame(path3)
INTC_return=(np.log(path3)-np.log(path3.shift(1))).dropna()


In [15]:
sim_return=pd.concat( [pd.Series(np.array(IBM_return).flatten()), pd.Series(np.array(MSFT_return).flatten()), pd.Series(np.array(INTC_return).flatten())], axis=1)
sim_return.columns=stocks.columns
sim_return

,IBM,MSFT,INTC
0,-0.015379,0.012673,0.032066
1,-0.018095,-0.015683,-0.009330
2,0.040375,0.038499,0.048700
3,0.006050,-0.039963,-0.017133
4,-0.013042,-0.003907,-0.027912
...,...,...,...
74999995,-0.061748,0.012675,-0.002048
74999996,-0.005182,0.003595,0.019544
74999997,-0.029439,-0.010495,-0.026492
74999998,0.018201,0.005579,0.012691


In [16]:
def MC_VaR(initial_investment, conf_level):
    MC_percentile95 = []
    for i, j in zip(sim_return.columns, range(len(sim_return.columns))):
        MC_percentile95.append(np.percentile(sim_return.loc[:, i], 5))
        print("Based on simulation 95% of {}'s return is {:.4f}"
              .format(i, MC_percentile95[j]))
        VaR_MC = (initial_investment - initial_investment * 
                  (1 + MC_percentile95[j]))
        print("Simulation VaR result for {} is {:.2f} "
              .format(i, VaR_MC))
        print('--' * 35)

In [17]:
MC_VaR(initial_investment, conf_level)

Based on simulation 95% of IBM's return is -0.0436
Simulation VaR result for IBM is 43604.26 
----------------------------------------------------------------------
Based on simulation 95% of MSFT's return is -0.0468
Simulation VaR result for MSFT is 46776.47 
----------------------------------------------------------------------
Based on simulation 95% of INTC's return is -0.0576
Simulation VaR result for INTC is 57636.88 
----------------------------------------------------------------------
